# SVM using Forest_fires

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("forestfires.csv")

# Data Pre-Processing

In [3]:
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [5]:
df.shape

(517, 31)

In [6]:
df.isnull().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [7]:
df["month"].unique()

array(['mar', 'oct', 'aug', 'sep', 'apr', 'jun', 'jul', 'feb', 'jan',
       'dec', 'may', 'nov'], dtype=object)

In [8]:
df["day"].unique()

array(['fri', 'tue', 'sat', 'sun', 'mon', 'wed', 'thu'], dtype=object)

In [9]:
df["month"].value_counts()

aug    184
sep    172
mar     54
jul     32
feb     20
jun     17
oct     15
apr      9
dec      9
jan      2
may      2
nov      1
Name: month, dtype: int64

In [10]:
#as per data there is less forest fire in nov ,jan and may

df["day"].value_counts()

sun    95
fri    85
sat    84
mon    74
tue    64
thu    61
wed    54
Name: day, dtype: int64

<AxesSubplot:ylabel='None'>

In [11]:
df.groupby('size_category').size().plot(kind='pie', autopct='%.2f')

In [12]:
#Graph shows there is small fire more than large fire
#The Support Vector Machine algorithm is effective for balanced classification
#let's  balanced the dataset

count_small , count_large =  df.size_category.value_counts()
count_large

139

In [13]:
target_count = df.size_category.value_counts()
target_count

small    378
large    139
Name: size_category, dtype: int64

In [14]:
print('Class small:', target_count['small'])
print('Class large:', target_count['large'])
print('Proportion:', round(target_count['large'] /  target_count['small'], 2), ': 1')

Class small: 378
Class large: 139
Proportion: 0.37 : 1


In [15]:
# Divide by class
df_class_small = df[df['size_category'] == 'small']
df_class_large = df[df['size_category'] == 'large']

In [16]:
df_class_small_under = df_class_small.sample(count_large)
df_test_under = pd.concat([df_class_small_under, df_class_large], axis=0)
print('Random under-sampling:')
print(df_test_under.size_category.value_counts())

Random under-sampling:
small    139
large    139
Name: size_category, dtype: int64


In [17]:
df_test_under

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
419,aug,sun,91.6,181.3,613.0,7.6,24.8,36,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small
275,dec,thu,84.6,26.4,352.0,2.0,5.1,61,4.9,0.0,...,0,0,0,0,0,0,0,0,0,small
83,aug,wed,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0,...,0,0,0,0,0,0,0,0,0,small
23,aug,sat,90.2,110.9,537.4,6.2,19.5,43,5.8,0.0,...,0,0,0,0,0,0,0,0,0,small
119,aug,thu,93.0,75.3,466.6,7.7,19.6,36,3.1,0.0,...,0,0,0,0,0,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,aug,tue,96.1,181.1,671.2,14.3,27.3,63,4.9,6.4,...,0,0,0,0,0,0,0,0,0,large
504,aug,wed,94.5,139.4,689.1,20.0,28.9,29,4.9,0.0,...,0,0,0,0,0,0,0,0,0,large
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large


# Feature Engineering

In [18]:
df_test_up = df_test_under.drop(["month" , "day"],axis = 1)
df_test_up

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
419,91.6,181.3,613.0,7.6,24.8,36,4.0,0.0,3.05,0,...,0,0,0,0,0,0,0,0,0,small
275,84.6,26.4,352.0,2.0,5.1,61,4.9,0.0,5.38,0,...,0,0,0,0,0,0,0,0,0,small
83,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small
23,90.2,110.9,537.4,6.2,19.5,43,5.8,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small
119,93.0,75.3,466.6,7.7,19.6,36,3.1,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,96.1,181.1,671.2,14.3,27.3,63,4.9,6.4,10.82,0,...,0,0,0,0,0,0,0,0,0,large
504,94.5,139.4,689.1,20.0,28.9,29,4.9,0.0,49.59,0,...,0,0,0,0,0,0,0,0,0,large
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,large
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,large


# Data Standardisation

In [19]:
X=df_test_up.iloc[:, :-1]
X.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
419,91.6,181.3,613.0,7.6,24.8,36,4.0,0.0,3.05,0,...,0,0,0,0,0,0,0,0,0,0
275,84.6,26.4,352.0,2.0,5.1,61,4.9,0.0,5.38,0,...,1,0,0,0,0,0,0,0,0,0
83,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,0
23,90.2,110.9,537.4,6.2,19.5,43,5.8,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,0
119,93.0,75.3,466.6,7.7,19.6,36,3.1,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from sklearn.preprocessing import LabelEncoder
y=df_test_up.iloc[:,-1]
# Encode label category
# small -> 1
# large -> 0
fire_encoder = LabelEncoder()
y = fire_encoder.fit_transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
# Scale the data to be between -1 and 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [22]:
X

array([[ 0.18993217,  0.98507333,  0.25550971, ..., -0.06008418,
        -0.14852213, -0.7090153 ],
       [-1.67395085, -1.30102261, -0.81038848, ..., -0.06008418,
        -0.14852213, -0.7090153 ],
       [ 0.32306667, -0.04949946,  0.42335805, ..., -0.06008418,
        -0.14852213, -0.7090153 ],
       ...,
       [-2.47275786, -0.85383922,  0.47032291, ..., -0.06008418,
        -0.14852213, -0.7090153 ],
       [-2.47275786, -0.85383922,  0.47032291, ..., -0.06008418,
        -0.14852213, -0.7090153 ],
       [-2.47275786, -0.85383922,  0.47032291, ..., -0.06008418,
        -0.14852213, -0.7090153 ]])

# Splitting dataset into training set and testing

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


In [24]:
from sklearn.svm import SVC
from sklearn import metrics

svc=SVC() #Default hyperparameters
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.5857142857142857


In [25]:
svc=SVC(kernel='rbf')
svc.fit(X_train,y_train)  #train on rbf kernal
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.5857142857142857


In [26]:
svc=SVC(kernel='linear')
svc.fit(X_train,y_train)    #train on linear kernal gives ggod accuracy than rbf
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.7428571428571429


In [27]:
svc=SVC(kernel='poly')
svc.fit(X_train,y_train)        #train on poly kernal
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.5571428571428572


# CV on Linear kernel

In [28]:
#Performing K-fold cross validation
from sklearn.model_selection import cross_val_score

svc=SVC(kernel='linear')
scores = cross_val_score(svc, X, y, cv=6, scoring='accuracy') #cv is cross validation
print(scores)

[0.53191489 1.         0.69565217 0.7173913  0.76086957 0.7826087 ]


In [29]:
print(scores.mean())

0.748072772124576


In [30]:
#Performing K-fold cross validation on rbf kernal

In [31]:
from sklearn.model_selection import cross_val_score
svc=SVC(kernel='rbf')
scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy') #cv is cross validation
print(scores)

[0.5        0.46428571 0.46428571 0.67857143 0.57142857 0.35714286
 0.39285714 0.53571429 0.33333333 0.37037037]


In [32]:
print(scores.mean())

0.46679894179894177


In [33]:
#it gives worst accuracy than linear
#So without finding an gamma value move to taking hyperparameter of class

In [34]:
svc=SVC(kernel='linear', C= 1)
svc.fit(X_train,y_train)    #train on linear kernal gives ggod accuracy than rbf
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.7428571428571429
